# Collect META5 Risk Variants Stats From Other GWAS
- **Author** - Frank Grenn
- **Date Started** - November 2019
- **Quick Description:** code to gather summary stats for the meta5 risk loci in other gwases. **Note** that this list is manually edited as the end to make sure maf is consistent  

 - #### 1) Age of Onset GWAS
 - #### 2) GBA Age of Onset GWAS
 - #### 3) GBA Modifier GWAS
 - #### 4) LRRK2 Modifier GWAS
 - #### 5) Asian GWAS
 - #### 6) META5 GWAS


In [ ]:
import pandas as pd
from scipy.stats import norm

In [ ]:
DATADIR ='$PATH/AppDataProcessing' 
WRKDIR = f"{DATADIR}/othersummarystats"

In [ ]:
writer = pd.ExcelWriter(f'{WRKDIR}/other_sum_stats.xlsx', engine='xlsxwriter')

#### get all the risk variants and loci

In [ ]:
meta5_loci = pd.read_csv(f"{DATADIR}/META5Loci.csv")
meta5_loci['GWAS']='META5'
print(meta5_loci.shape)
print(meta5_loci.head())

In [ ]:
prog_loci = pd.read_csv(f"{DATADIR}/ProgressionLoci.csv")
prog_loci['GWAS']='Progression'
prog_loci['EFFECT_ALLELE']=prog_loci['ALT']
prog_loci['OTHER_ALLELE']=prog_loci['REF']
prog_loci['P_COJO']=None
prog_loci = prog_loci.rename(columns={"MAF": "FREQ"})
prog_loci['EFFECT_FREQ']=prog_loci['FREQ']

print(prog_loci.shape)
print(prog_loci.head())

In [ ]:
asian_loci = pd.read_csv(f"{DATADIR}/AsianLoci.csv")
asian_loci['GWAS']='Asian'
asian_loci['EFFECT_ALLELE']=asian_loci['ALT']
asian_loci['OTHER_ALLELE']=asian_loci['REF']
asian_loci['P_COJO']=None
asian_loci = asian_loci.rename(columns={"MAF": "FREQ"})
asian_loci['EFFECT_FREQ']=asian_loci['FREQ']
print(asian_loci.shape)
print(asian_loci.head())

In [ ]:
loci = meta5_loci.append(prog_loci).append(asian_loci)
print(loci.shape)
print(loci.head())

In [ ]:
loci_sub = loci[['RSID','CHR','BP','CHR_BP','EFFECT_FREQ','EFFECT_ALLELE','OTHER_ALLELE']]

## 1) Age of Onset GWAS

In [ ]:
aoo = pd.read_csv("$PATH/summary_stats/age_of_onset/sorted_AAO_april3_18_final_all_data.txt", sep="\t")
print(aoo.shape)
print(aoo.head())

In [ ]:
print(list(aoo.columns.values))

In [ ]:
aoo_sub = aoo[['MarkerName','Allele1','Allele2','Freq1','Effect','StdErr','P-value']]

In [ ]:
loci_sub['MarkerName'] = 'chr' + loci_sub['CHR_BP']
print(loci_sub.head())

In [ ]:
merge_aoo = pd.merge(loci_sub, aoo_sub, on='MarkerName',how='left')
print(merge_aoo.shape)
print(merge_aoo.head())

#### now check for the right allele
looks like Allele1 from aoo should match EFFECT_ALLELE

In [ ]:
aoo_match=merge_aoo[merge_aoo['EFFECT_ALLELE']==merge_aoo['Allele1'].str.upper()]
print(aoo_match.shape)
print(aoo_match.head())

In [ ]:
aoo_mismatch=merge_aoo[merge_aoo['EFFECT_ALLELE']!=merge_aoo['Allele1'].str.upper()]
print(aoo_mismatch.shape)
print(aoo_mismatch.head())

In [ ]:
aoo_mismatch['Allele1']=aoo_mismatch['EFFECT_ALLELE']
aoo_mismatch['Allele2']=aoo_mismatch['OTHER_ALLELE']
aoo_mismatch['Freq1']=1-aoo_mismatch['Freq1']
aoo_mismatch['Effect']=-1*aoo_mismatch['Effect']


In [ ]:
final_aoo=aoo_mismatch.append(aoo_match)
print(final_aoo.shape)
print(final_aoo.head())

In [ ]:
final_aoo = final_aoo[['RSID','CHR','BP','CHR_BP','Freq1','Effect','StdErr','P-value']]
final_aoo.columns = ['RSID','CHR','BP','CHR_BP','EFFECT_FREQ','BETA','SE','P']

In [ ]:
print(final_aoo.shape)
print(final_aoo.head())

In [ ]:
final_aoo.to_excel(writer, sheet_name='AAO',index=None)

## 2) GBA Age of Onset GWAS

In [ ]:
gba_aoo = pd.read_csv("$PATH/summary_stats/GBA_GWAS/HARDCALL_GBA_case_age.SingleWald_MAF.assoc", sep="\t")
print(gba_aoo.shape)
print(gba_aoo.head())

In [ ]:
gba_aoo_sub = gba_aoo[['Test','A1','A2','CHROM','POS','Beta','SE','Pvalue','MAF']]
print(gba_aoo_sub.shape)

In [ ]:
merge_gba_aoo = pd.merge(loci_sub, gba_aoo_sub, how='left', left_on='CHR_BP', right_on='Test')
print(merge_gba_aoo.shape)
print(merge_gba_aoo.head())

#### now check for the right allele
looks like A1 from gba_aoo should match EFFECT_ALLELE

In [ ]:
gba_aoo_match=merge_gba_aoo[merge_gba_aoo['EFFECT_ALLELE']==merge_gba_aoo['A1']]
print(gba_aoo_match.shape)
print(gba_aoo_match.head())

In [ ]:
gba_aoo_mismatch=merge_gba_aoo[merge_gba_aoo['EFFECT_ALLELE']!=merge_gba_aoo['A1']]
print(gba_aoo_mismatch.shape)
print(gba_aoo_mismatch.head())

In [ ]:
gba_aoo_mismatch['MAF']=1-gba_aoo_mismatch['MAF']
gba_aoo_mismatch['Beta']=-1*gba_aoo_mismatch['Beta']

In [ ]:
final_gba_aoo=gba_aoo_mismatch.append(gba_aoo_match)
print(final_gba_aoo.shape)
print(final_gba_aoo.head())

In [ ]:
final_gba_aoo = final_gba_aoo[['RSID','CHR','BP','CHR_BP','MAF','Beta','SE','Pvalue']]
final_gba_aoo.columns = ['RSID','CHR','BP','CHR_BP','EFFECT_FREQ','BETA','SE','P']

In [ ]:
final_gba_aoo.to_excel(writer, sheet_name='GBA_AAO',index=None)

## 3) GBA Modifier GWAS

In [ ]:
meta_gba = pd.read_csv("$PATH/summary_stats/GBA_GWAS/META_GBA_penetrance_modifier_carriers_GWAS.txt", sep="\t")
print(meta_gba.shape)
print(meta_gba.head())

In [ ]:
meta_gba_sub = meta_gba[['MarkerName','Allele1','Allele2','Freq1','StdErr','Effect','P-value']]
print(meta_gba_sub.shape)

In [ ]:
merge_gba = pd.merge(loci_sub, meta_gba_sub, how='left', left_on='CHR_BP', right_on='MarkerName')
print(merge_gba.shape)
print(merge_gba.head())

#### now check for the right allele
looks like Allele1 from gba shoul match EFFECT_ALLELE

In [ ]:
gba_match=merge_gba[merge_gba['EFFECT_ALLELE']==merge_gba['Allele1'].str.upper()]
print(gba_match.shape)
print(gba_match.head())

In [ ]:
gba_mismatch=merge_gba[merge_gba['EFFECT_ALLELE']!=merge_gba['Allele1'].str.upper()]
print(gba_mismatch.shape)
print(gba_mismatch.head())

In [ ]:
gba_mismatch['Freq1']=1-gba_mismatch['Freq1']
gba_mismatch['Effect']=-1*gba_mismatch['Effect']

In [ ]:
final_gba = gba_match.append(gba_mismatch)
print(final_gba.shape)
print(final_gba.head())

In [ ]:
final_gba = final_gba[['RSID','CHR','BP','CHR_BP','Freq1','Effect','StdErr','P-value']]
final_gba.columns = ['RSID','CHR','BP','CHR_BP','EFFECT_FREQ','BETA','SE','P']

In [ ]:

print(final_gba.head())

In [ ]:
final_gba.to_excel(writer, sheet_name='GBA_mod',index=None)

## 4) LRRK2 Modifier GWAS

In [ ]:
lrrk2 = pd.read_csv(f"{WRKDIR}/LRRK2_GWAS_risk_Variants.csv")
print(lrrk2.shape)
print(lrrk2.head())

In [ ]:
lrrk2['name']=lrrk2['name'].astype(str).str[:-2]
print(lrrk2.head())

In [ ]:
merge_lrrk2 = pd.merge(loci_sub, lrrk2, how='left', left_on='RSID', right_on='name')
print(merge_lrrk2.shape)
print(merge_lrrk2.head())

#### now check for the right allele
looks like A1 from lrrk2 shoul match EFFECT_ALLELE

In [ ]:
lrrk2_match=merge_lrrk2[merge_lrrk2['EFFECT_ALLELE']==merge_lrrk2['A1']]
print(lrrk2_match.shape)
print(lrrk2_match.head())

In [ ]:
lrrk2_mismatch=merge_lrrk2[merge_lrrk2['EFFECT_ALLELE']!=merge_lrrk2['A1']]
print(lrrk2_mismatch.shape)
print(lrrk2_mismatch.head())

In [ ]:
lrrk2_mismatch['A1_freq']=1-lrrk2_mismatch['A1_freq']
lrrk2_mismatch['beta']=-1*lrrk2_mismatch['beta']

In [ ]:
final_lrrk2 = lrrk2_match.append(lrrk2_mismatch)
print(final_lrrk2.shape)
print(final_lrrk2.head())

In [ ]:
final_lrrk2 = final_lrrk2[['RSID','CHR','BP','CHR_BP','A1_freq','beta','std.error','p.value']]
final_lrrk2.columns = ['RSID','CHR','BP','CHR_BP','EFFECT_FREQ','BETA','SE','P']

In [ ]:
print(final_lrrk2.head())

In [ ]:
final_lrrk2.to_excel(writer, sheet_name='LRRK2',index=None)

## 5) Asian GWAS

In [ ]:
#6724PDcases-24851controls-5843213snps-summary-stats-maf5-metaP-SE.txt.gz
#6724PDcases-24851controls-5843213snps-summary-stats-metaP-SE.txt.gz
#6724PDcases-24851controls-5843213snps-summary-stats-metaP.txt.gz
#779PDcases-13227controls-2402394snps-hmg2017-summary-stats-OR-SE.txt
#Asian_GWAS_2019.gz

asian_gwas = pd.read_csv(f"{DATADIR}/asiangwas_sumstats_harmonized.csv")
print(asian_gwas.shape)
print(asian_gwas.head())


In [ ]:
asian_gwas_sub = asian_gwas[['RSID','A1','A2','BETA','SE','FREQ','P']]

In [ ]:
loci_sub = loci[['RSID','CHR','BP','CHR_BP','EFFECT_FREQ','EFFECT_ALLELE','OTHER_ALLELE','BETA']]

In [ ]:
merge_asian_gwas = pd.merge(loci_sub, asian_gwas_sub, how='left', left_on='RSID', right_on='RSID')
print(merge_asian_gwas.shape)
print(merge_asian_gwas.tail())

In [ ]:
merge_asian_gwas

#### now check for the right allele
need to look at beta since we don't have freqs, looks like asian gwas A1 should match EFFECT_ALLELE

In [ ]:
asian_gwas_match=merge_asian_gwas[merge_asian_gwas['EFFECT_ALLELE']==merge_asian_gwas['A1']]
print(asian_gwas_match.shape)
print(asian_gwas_match.head())

In [ ]:
asian_gwas_mismatch=merge_asian_gwas[merge_asian_gwas['EFFECT_ALLELE']!=merge_asian_gwas['A1']]
print(asian_gwas_mismatch.shape)
print(asian_gwas_mismatch.tail())

In [ ]:
asian_gwas_mismatch['BETA_y']=-1*asian_gwas_mismatch['BETA_y']

In [ ]:
final_asian_gwas = asian_gwas_match.append(asian_gwas_mismatch)
print(final_asian_gwas.shape)
print(final_asian_gwas.head())

In [ ]:
#for now no frequencies are available so set to NA
final_asian_gwas['maf']=None

#### (optional) recalculate the SE if missing

P = 2 * pnorm(-ABS(Z))  
Z = beta/SE  
in python:  
P = 2 * norm.cdf(-abs(beta/SE))  
or  
SE=beta/norm.ppf(P/2)

In [ ]:
no_se = final_asian_gwas[final_asian_gwas['SE']=='#NUM!']
no_se

In [ ]:
rsid_to_calc = 'rs11557080'#rs11557080

In [ ]:
beta = final_asian_gwas[final_asian_gwas['RSID']==rsid_to_calc]['BETA_y']
P = final_asian_gwas[final_asian_gwas['RSID']==rsid_to_calc]['P']
final_asian_gwas['SE'][final_asian_gwas['RSID']==rsid_to_calc] = abs(beta/-norm.ppf(P/2))

In [ ]:
final_asian_gwas[final_asian_gwas['RSID']==rsid_to_calc]

In [ ]:
final_asian_gwas = final_asian_gwas[['RSID','CHR','BP','CHR_BP','maf','BETA_y','SE','P']]
final_asian_gwas.columns = ['RSID','CHR','BP','CHR_BP','EFFECT_FREQ','BETA','SE','P']

In [ ]:

print(final_asian_gwas.shape)
print(final_asian_gwas.tail(10))


In [ ]:
final_asian_gwas.to_excel(writer, sheet_name='Asian_GWAS',index=None)

## 6) META5 GWAS

In [ ]:
meta5_gwas = pd.read_csv(f"{DATADIR}/meta5_sumstats_harmonized.csv")
print(meta5_gwas.shape)
print(meta5_gwas.head())

In [ ]:
loci_sub.columns

In [ ]:
meta5_gwas_sub = meta5_gwas[['RSID','REF','ALT','A1','A2','BETA','SE','FREQ','P']]

In [ ]:
loci_sub = loci[['RSID','CHR','BP','CHR_BP','EFFECT_FREQ','EFFECT_ALLELE','OTHER_ALLELE']]

In [ ]:
merge_meta5_gwas = pd.merge(loci_sub, meta5_gwas_sub, how='left', left_on='RSID', right_on='RSID')
print(merge_meta5_gwas.shape)
print(merge_meta5_gwas.tail())

In [ ]:
print(merge_meta5_gwas.head())

#### now check for the right allele

In [ ]:
meta5_gwas_match=merge_meta5_gwas[merge_meta5_gwas['EFFECT_ALLELE']==merge_meta5_gwas['A1']]
print(meta5_gwas_match.shape)
print(meta5_gwas_match.head())

In [ ]:
meta5_gwas_mismatch=merge_meta5_gwas[merge_meta5_gwas['EFFECT_ALLELE']!=merge_meta5_gwas['A1']]
print(meta5_gwas_mismatch.shape)
print(meta5_gwas_mismatch.tail())

In [ ]:
meta5_gwas_mismatch['BETA']=-1*meta5_gwas_mismatch['BETA']
meta5_gwas_mismatch['FREQ']=1-meta5_gwas_mismatch['FREQ']

In [ ]:
final_meta5_gwas = meta5_gwas_match.append(meta5_gwas_mismatch)

print(final_meta5_gwas.shape)
print(final_meta5_gwas.head())

In [ ]:
final_meta5_gwas = final_meta5_gwas[['RSID','CHR','BP','CHR_BP','FREQ','BETA','SE','P']]
final_meta5_gwas.columns = ['RSID','CHR','BP','CHR_BP','EFFECT_FREQ','BETA','SE','P']

In [ ]:
final_meta5_gwas.to_excel(writer, sheet_name='META5',index=None)

## 7) Jansen et al 2019 Alzheimer's Disease GWAS

In [ ]:

#ad = pd.read_csv('$PATH/summary_stats/Jansen_2019/AD_sumstats_Jansenetal.txt.gz', sep='\t| ')
ad = pd.read_csv('$PATH/AppDataProcessing/othersummarystats/AD_sumstats_Jansenetal_2019sept.txt.gz', sep='\t| ')
print(ad.shape)
print(ad.head())

In [ ]:
colnames = [name for name in ad.columns if 'Unnamed' not in name]
colnames

In [ ]:
ad_temp = ad.iloc[:,0:14]
ad_temp.columns = colnames
ad_temp.head()

In [ ]:
ad_sub = ad[['SNP','A1','A2','EAF','BETA','SE','P']]

In [ ]:
print(loci_sub.head())

In [ ]:
merge_ad = pd.merge(loci_sub, ad_sub, left_on = 'RSID', right_on = 'SNP',how='left')
print(merge_ad.shape)
print(merge_ad.head())

#### AD sum stats are already calculated by minor allele, so nothing extra needs to be done

In [ ]:
final_ad = merge_ad[['RSID','CHR','BP','CHR_BP','EAF','BETA','SE','P']]
final_ad.columns = ['RSID','CHR','BP','CHR_BP','EFFECT_FREQ','BETA','SE','P']

In [ ]:
print(final_ad.shape)
print(final_ad.head())



In [ ]:
final_ad.to_excel(writer, sheet_name='AD',index=None)

## 8) Nicolas et al. 2018 Amyotrophic Lateral Sclerosis GWAS

In [ ]:
als = pd.read_csv("$PATH/summary_stats/Nicolas_2017/alsMetaSummaryStats_march21st2018.tab.zip", sep="\t")
print(als.shape)
print(als.head())


In [ ]:
als_sub = als[['SNP','Allele1','Allele2','effectAlleleFreq','Effect','StdErr','P']]

In [ ]:
merge_als = pd.merge(loci_sub, als_sub, left_on='CHR_BP',right_on = 'SNP',how='left')
print(merge_als.shape)
print(merge_als.head())

#### now check for the right allele
looks like Allele1 from als should match EFFECT_ALLELE

In [ ]:
als_match=merge_als[merge_als['EFFECT_ALLELE']==merge_als['Allele1'].str.upper()]
print(als_match.shape)
print(als_match.head())

In [ ]:
als_mismatch=merge_als[merge_als['EFFECT_ALLELE']!=merge_als['Allele1'].str.upper()]
print(als_mismatch.shape)
print(als_mismatch.head())

In [ ]:
als_mismatch['Allele1']=als_mismatch['EFFECT_ALLELE']
als_mismatch['Allele2']=als_mismatch['OTHER_ALLELE']
als_mismatch['effectAlleleFreq']=1-als_mismatch['effectAlleleFreq']
als_mismatch['Effect']=-1*als_mismatch['Effect']

In [ ]:
final_als=als_mismatch.append(als_match)
print(final_als.shape)
print(final_als.head())

In [ ]:
final_als = final_als[['RSID','CHR','BP','CHR_BP','effectAlleleFreq','Effect','StdErr','P']]
final_als.columns = ['RSID','CHR','BP','CHR_BP','EFFECT_FREQ','BETA','SE','P']

In [ ]:
print(final_als.shape)
print(final_als.head())



In [ ]:
final_als.to_excel(writer, sheet_name='ALS',index=None)

## 9) Loesch et al. 2020 Latin America GWAS

In [ ]:
#reading in hg38
la = pd.read_csv("$PATH/LARGE_PD/large.assoc_single.ALL.04302020.txt", sep="\t")
print(la.shape)
print(la.head())


In [ ]:
la['chrbp']=la['#SNP'].str.split(":").str[0].str.replace("chr","")+":"+la['#SNP'].str.split(":").str[1]
print(la.head())

In [ ]:
#read in hg19 conversion coordinates
la_conv = pd.read_table("$PATH/AppDataProcessing/othersummarystats/large_gwas_all_pos_only.hg19.noUnmap.tsv",header=None)
la_conv.columns = ['chr_38','bp_38','drop1','drop2','chr_19','bp_19','drop3']
la_conv['chrbp38']=la_conv['chr_38'].astype(str)+":"+la_conv['bp_38'].astype(str)
la_conv['chrbp19']=la_conv['chr_19'].astype(str)+":"+la_conv['bp_19'].astype(str)
la_conv = la_conv[['chrbp38','chrbp19']]

print(la_conv.shape)
print(la_conv.head())


In [ ]:
merge_pos = pd.merge(left = la, right = la_conv, left_on = "chrbp", right_on = "chrbp38", how = "left")
print(merge_pos.shape)
print(merge_pos.head())

In [ ]:
merge_pos['SE']=1/merge_pos['Score.SE']
print(merge_pos.head())

In [ ]:
la_sub = merge_pos[['chrbp19','ref','alt','freq','beta','SE','Score.pval']]

In [ ]:
loci_sub.head()

In [ ]:
merge_la = pd.merge(loci_sub, la_sub, left_on='CHR_BP',right_on = 'chrbp19',how='left')
print(merge_la.shape)
print(merge_la.head())

In [ ]:
merge_la.head(20)

In [ ]:
merge_la.tail(20)

#### now check for the right allele
looks like ref from la should match OTHER_ALLELE

In [ ]:
la_match=merge_la[merge_la['OTHER_ALLELE']==merge_la['ref'].str.upper()]
print(la_match.shape)
print(la_match.head())

In [ ]:
la_mismatch=merge_la[merge_la['OTHER_ALLELE']!=merge_la['ref'].str.upper()]
print(la_mismatch.shape)
print(la_mismatch.head())

In [ ]:
la_mismatch['freq']=1-la_mismatch['freq']
la_mismatch['beta']=-1*la_mismatch['beta']

In [ ]:
final_la=la_mismatch.append(la_match)
print(final_la.shape)
print(final_la.head())

In [ ]:
final_la = final_la[['RSID','CHR','BP','CHR_BP','freq','beta','SE','Score.pval']]
final_la.columns = ['RSID','CHR','BP','CHR_BP','EFFECT_FREQ','BETA','SE','P']

In [ ]:
print(final_la.shape)
print(final_la.head())


In [ ]:
final_la.to_csv("$PATH/AppDataProcessing/othersummarystats/LATAM.csv",index=None)

In [ ]:
final_la.to_excel(writer, sheet_name='LATAM',index=None)

## 10) Blauwendraat et al. 2021 Male GWAS and Female GWAS

In [ ]:
#reading in hg38
#sexgwas = pd.read_csv("/$PATH/summary_stats/sex_specific/FEMALE/FEMALE_PD_filtered_sumstats_no_multi_allelics_RSID.txt", sep="\t")
sexgwas = pd.read_csv("$PATH/summary_stats/sex_specific/MALE/MALE_PD_filtered_sumstats_no_multi_allelics_RSID.txt", sep="\t")
print(sexgwas.shape)
print(sexgwas.head())

In [ ]:
sexgwas_sub = sexgwas[['ID','REF','ALT','MarkerName','Freq1','Effect','StdErr','P-value','Allele1','Allele2']]

In [ ]:
sexgwas_sub.head()

In [ ]:
merge_sexgwas = pd.merge(loci_sub, sexgwas_sub, left_on='CHR_BP',right_on = 'MarkerName',how='left')
print(merge_sexgwas.shape)
print(merge_sexgwas.head())

In [ ]:
merge_sexgwas.tail()

#### now check for the right allele
looks like Allele1 from sex gwas should match EFFECT_ALLELE

In [ ]:
sexgwas_match=merge_sexgwas[merge_sexgwas['EFFECT_ALLELE']==merge_sexgwas['Allele1'].str.upper()]
print(sexgwas_match.shape)
print(sexgwas_match.head())

In [ ]:
sexgwas_mismatch=merge_sexgwas[merge_sexgwas['EFFECT_ALLELE']!=merge_sexgwas['Allele1'].str.upper()]
print(sexgwas_mismatch.shape)
print(sexgwas_mismatch.head())

In [ ]:
sexgwas_mismatch['Freq1']=1-sexgwas_mismatch['Freq1']
sexgwas_mismatch['Effect']=-1*sexgwas_mismatch['Effect']

In [ ]:
final_sexgwas=sexgwas_mismatch.append(sexgwas_match)
print(final_sexgwas.shape)
print(final_sexgwas.head())

In [ ]:
final_sexgwas = final_sexgwas[['RSID','CHR','BP','CHR_BP','Freq1','Effect','StdErr','P-value']]
final_sexgwas.columns = ['RSID','CHR','BP','CHR_BP','EFFECT_FREQ','BETA','SE','P']

In [ ]:
print(final_sexgwas.shape)
print(final_sexgwas.head())


In [ ]:
#final_sexgwas.to_csv("$PATH/AppDataProcessing/othersummarystats/Female_PD_GWAS.csv",index=None)
final_sexgwas.to_csv("$PATH/AppDataProcessing/othersummarystats/Male_PD_GWAS.csv",index=None)

In [ ]:
#final_sexgwas.to_excel(writer, sheet_name='Female_PD',index=None)
final_sexgwas.to_excel(writer, sheet_name='Male_PD',index=None)

## 11) Chia et al. 2021 Lewy Body Dementia GWAS

In [ ]:
#reading in hg38
#sexgwas = pd.read_csv("$PATH/summary_stats/sex_specific/FEMALE/FEMALE_PD_filtered_sumstats_no_multi_allelics_RSID.txt", sep="\t")
lbdgwas = pd.read_csv("$PATH/AppDataProcessing/othersummarystats/LBD_GWAS_GCST90001390_buildGRCh38.tsv.gz", sep="\t")
print(lbdgwas.shape)
print(lbdgwas.head())

In [ ]:
loci_sub.head()

In [ ]:
merge_lbdgwas = pd.merge(loci_sub, lbdgwas, left_on='RSID',right_on = 'variant_id',how='left')
print(merge_lbdgwas.shape)
print(merge_lbdgwas.head())

In [ ]:
merge_lbdgwas.tail()

In [ ]:
merge_lbdgwas.iloc[5:15,:]

#### now check for the right allele
looks like effect_allele from lbd gwas should match EFFECT_ALLELE

In [ ]:
lbdgwas_match=merge_lbdgwas[merge_lbdgwas['EFFECT_ALLELE']==merge_lbdgwas['effect_allele'].str.upper()]
print(lbdgwas_match.shape)
print(lbdgwas_match.head())

In [ ]:
lbdgwas_mismatch=merge_lbdgwas[merge_lbdgwas['EFFECT_ALLELE']!=merge_lbdgwas['effect_allele'].str.upper()]
print(lbdgwas_mismatch.shape)
print(lbdgwas_mismatch.head())

In [ ]:
lbdgwas_mismatch['effect_allele_frequency']=1-lbdgwas_mismatch['effect_allele_frequency']
lbdgwas_mismatch['beta']=-1*lbdgwas_mismatch['beta']

In [ ]:
final_lbdgwas=lbdgwas_mismatch.append(lbdgwas_match)
print(final_lbdgwas.shape)
print(final_lbdgwas.head())

In [ ]:
final_lbdgwas = final_lbdgwas[['RSID','CHR','BP','CHR_BP','effect_allele_frequency','beta','standard_error','p_value']]
final_lbdgwas.columns = ['RSID','CHR','BP','CHR_BP','EFFECT_FREQ','BETA','SE','P']

In [ ]:
print(final_lbdgwas.shape)
print(final_lbdgwas.head())

In [ ]:
#final_sexgwas.to_csv("$PATH/AppDataProcessing/othersummarystats/Female_PD_GWAS.csv",index=None)
final_lbdgwas.to_csv("$PATH/AppDataProcessing/othersummarystats/LBD_GWAS.csv",index=None)

In [ ]:
#final_sexgwas.to_excel(writer, sheet_name='Female_PD',index=None)
final_lbdgwas.to_excel(writer, sheet_name='LBD_GWAS',index=None)

In [ ]:
writer.save()